In [1]:
#!/usr/bin/env python
import sys
import pandas as pd
import pymongo
import json
import os
import numpy

# Extracting Terrorist Data from CSV file

In [2]:
file = 'globalterrorismdb_0718dist.csv'
terror_df = pd.read_csv(file, encoding = "ISO-8859-1")
low_memory=False,
terror_df.head()

C:\Users\azizi\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


# Dashboard Totals

In [3]:
dashboard_df = terror_df.loc[:, ["eventid","nkill","nwound","success","region","country_txt","iyear","region_txt",]]
dashboard_df.head()

,eventid,nkill,nwound,success,region,country_txt,iyear,region_txt
0,197000000001,1.0,0.0,1,2,Dominican Republic,1970,Central America & Caribbean
1,197000000002,0.0,0.0,1,1,Mexico,1970,North America
2,197001000001,1.0,0.0,1,5,Philippines,1970,Southeast Asia
3,197001000002,NaN,NaN,1,8,Greece,1970,Western Europe
4,197001000003,NaN,NaN,1,4,Japan,1970,East Asia


In [4]:
#Finding Totals
total_attacks = int(dashboard_df["eventid"].count())
total_wounded = int(dashboard_df["nwound"].count())
total_fatalities = int( dashboard_df["nkill"].sum())
total_success =dashboard_df["success"].loc[dashboard_df["success"]==1].count()
print(f"Total Attacks : {total_attacks}")
print(f"Total Wounded : {total_wounded}")
print(f"Total Fatalities : {total_fatalities}")
print(f"Total Success : {total_success}")

Total Attacks : 181691
Total Wounded : 165380
Total Fatalities : 411868
Total Success : 161632


In [5]:
# Creating a DataFrame for Totals
dashboard_table = pd.DataFrame({"total_attacks": [total_attacks], "total_fatalities": [total_fatalities], 'total_wounded': [total_wounded], 'total_success':[total_success] })
#To be loaded to MongoDB
dashboard_table

,total_attacks,total_fatalities,total_wounded,total_success
0,181691,411868,165380,161632


# Stats for Regions

In [6]:
region_values = dashboard_df["region_txt"].value_counts()

In [7]:
region_sum_df = pd.DataFrame(region_values)
region_sum_df.rename(columns={"region_txt":"total_terror"}, inplace=True)
#To be loaded to MongDB
regstats_df = region_sum_df.reset_index()
regstats_df

,index,total_terror
0,Middle East & North Africa,50474
1,South Asia,44974
2,South America,18978
3,Sub-Saharan Africa,17550
4,Western Europe,16639
5,Southeast Asia,12485
6,Central America & Caribbean,10344
7,Eastern Europe,5144
8,North America,3456
9,East Asia,802


# Incident Data(yearly)

In [8]:
incident_pyear = dashboard_df['iyear'].value_counts()

In [9]:
incident_pyear_df = pd.DataFrame(incident_pyear)
incident_pyear_df.rename(columns={"iyear":"incidents"}, inplace=True)

#To be loaded to MongoDB
incident_df = incident_pyear_df.reset_index()
incident_df.head()

,index,incidents
0,2014,16903
1,2015,14965
2,2016,13587
3,2013,12036
4,2017,10900


# Data for Terrorist Risk Map

In [10]:
country_values = dashboard_df["country_txt"].value_counts()
country_bins = pd.DataFrame(country_values)
country_bins.rename(columns={"country_txt":"incidents"}, inplace=True)
country_bins.head()

,incidents
Iraq,24636
Pakistan,14368
Afghanistan,12731
India,11960
Colombia,8306


In [11]:
bins = [0,1000,3000, 24636]
group_names = ["Green", "Yellow", "Red"]
country_bins["danger_color"] = pd.cut(country_bins["incidents"], bins, labels=group_names)
country_bins = country_bins.reset_index()

In [12]:
country_danger_color = pd.DataFrame({'country' : country_bins['index'], 'incidents': country_bins['incidents'], 'danger_color': country_bins['danger_color']})
#To be loaded to MongoDB
country_danger_color.head()

,country,incidents,danger_color
0,Iraq,24636,Red
1,Pakistan,14368,Red
2,Afghanistan,12731,Red
3,India,11960,Red
4,Colombia,8306,Red


# LINE CHART DATA Table

In [13]:
df = terror_df
data = df.groupby(['iyear', 'country_txt']).count()
data.head()

eventid  imonth  iday  approxdate  extended  resolution  \
iyear country_txt                                                            
1970  Argentina         21      21    21           0        21           2   
      Australia          1       1     1           0         1           0   
      Belgium            1       1     1           0         1           0   
      Bolivia            1       1     1           0         1           1   
      Brazil             6       6     6           0         6           3   

                   country  region  region_txt  provstate   ...     addnotes  \
iyear country_txt                                           ...                
1970  Argentina         21      21          21         21   ...            0   
      Australia          1       1           1          1   ...            0   
      Belgium            1       1           1          1   ...            0   
      Bolivia            1       1           1          1   ...            0   
      Brazil             6       6           6          6   ...            0   

                   scite1  scite2  scite3  dbsource  INT_LOG  INT_IDEO  \
iyear country_txt                                                        
1970  Argentina         0       0       0        21       21        21   
      Australia         0       0       0         1        1         1   
      Belgium           0       0       0         1        1         1   
      Bolivia           0       0       0         1        1         1   
      Brazil            0       0       0         6        6         6   

                   INT_MISC  INT_ANY  related  
iyear country_txt                              
1970  Argentina          21       21        0  
      Australia           1        1        0  
      Belgium             1        1        0  
      Bolivia             1        1        0  
      Brazil              6        6        0  

[5 rows x 133 columns]

In [14]:
df = data.reset_index(level=['iyear', 'country_txt'])
df_index = df.sort_values(by=['country_txt', 'iyear'])

table = df_index[['iyear','country_txt', 'eventid']].reset_index()
table['value_1'] = table.country_txt.shift()

answer=[]
total=0

for i in range(len(table)):
    if table['country_txt'][i] == table['value_1'][i]:
        newtotal = table['eventid'][i]+total
        total=newtotal
    else:
        newtotal=table['eventid'][i]
        total=table['eventid'][i]
    answer.append(newtotal)      

table['freq'] = answer
table =table[['iyear','country_txt','freq']].sort_values(by=['country_txt','iyear'])

final_table = pd.DataFrame({'name': table['country_txt'], 'time': table['iyear'], 'event': table['freq']})
#To be loaded to MongoDB
final_table.head(5) 

,name,time,event
0,Afghanistan,1973,1
1,Afghanistan,1979,4
2,Afghanistan,1987,5
3,Afghanistan,1988,16
4,Afghanistan,1989,26


# Pushing Data to MongoDB

In [42]:
#Connecting to Mongo DB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
#------------------------------------------------------------------------------------------#
# To Load Dashboard Totals    
def totals_toMongodb():
    final_df = dashboard_table
    db = client.Terrorist
    collection = db.DashTotals
    collection.delete_many({})
    for index, row in final_df.iterrows():
        post = {'total_attacks': int(row['total_attacks']),'total_fatalities': int(row['total_fatalities']),'total_wounded': int(row['total_wounded']),'total_success':int(row['total_success'])}
        collection.insert_one(post)
#------------------------------------------------------------------------------------------#        
#to Load Regional Stats
def regionalStats_toMongodb():
    final_df = regstats_df
    db = client.Terrorist
    collection = db.RegStats
    collection.delete_many({})
    for index, row in final_df.iterrows():
        post = {
            'regions': row['index'], 'total_terror': row['total_terror']
        }
        collection.insert_one(post)
#------------------------------------------------------------------------------------------#
#To Load YEarly Incidents
def incidents_toMongodb():
    final_df = incident_df
    db = client.Terrorist
    collection = db.IncidentsYearly
    collection.delete_many({})
    for index, row in final_df.iterrows():
        post = {
            'year': int(row['index']), 'incidents': int(row['incidents'])
        }
        collection.insert_one(post)
#------------------------------------------------------------------------------------------#
# To Load LineChart
def line_chart_toMongodb():
    final_df = final_table
    db = client.Terrorist
    collection = db.LineChart
    collection.delete_many({})
    for index, row in final_df.iterrows():
        post = {
            'name': row['name'], 'time': row['time'], 'event': row['event']
        }
        collection.insert_one(post)
#------------------------------------------------------------------------------------------#
# To Load RiskMap
def riskMap_toMongodb():
    final_df = country_danger_color
    db = client.Terrorist
    collection = db.RiskMap
    collection.delete_many({})
    for index, row in final_df.iterrows():
        post = {
           'country' : row['country'], 'incidents': row['incidents'], 'danger_color': row['danger_color']
        }
        collection.insert_one(post)



In [43]:
#Functions to create/load collections and data.
totals_toMongodb()
regionalStats_toMongodb()
incidents_toMongodb()
line_chart_toMongodb()
riskMap_toMongodb()